In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Dropout, ReLU, Concatenate
from tensorflow.keras.models import Model
import os
import numpy as np
from tensorflow.keras.optimizers import Adam
import glob
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Helper function to load and preprocess images
def load_image(image_file, size=(256, 256)):
    image = Image.open(image_file)
    image = image.resize(size)
    image = np.array(image).astype('float16') / 127.5 - 1.0  # Normalize to [-1, 1]
    return image

def load_dataset(original_dir, cartoonized_dir, size=(256, 256)):
    original_images = sorted(glob.glob(os.path.join(original_dir, "*.jpg")))
    cartoonized_images = sorted(glob.glob(os.path.join(cartoonized_dir, "*.jpg")))
    
    originals = [load_image(f, size) for f in original_images]
    cartoonized = [load_image(f, size) for f in cartoonized_images]
            
            # Convertir en tenseurs avant de retourner
    yield tf.convert_to_tensor(np.array(originals), dtype=tf.float32), \
              tf.convert_to_tensor(np.array(cartoonized), dtype=tf.float32)
    #return 
    return np.array(originals), np.array(cartoonized)

# Build the Generator (U-Net)
def build_generator():
    inputs = Input(shape=[256, 256, 3])

    # Encoder
    down_stack = [
        Conv2D(64, (4, 4), strides=2, padding="same"),
        Conv2D(128, (4, 4), strides=2, padding="same"),
        Conv2D(256, (4, 4), strides=2, padding="same"),
        Conv2D(512, (4, 4), strides=2, padding="same"),
        Conv2D(512, (4, 4), strides=2, padding="same"),
        Conv2D(512, (4, 4), strides=2, padding="same"),
        Conv2D(512, (4, 4), strides=2, padding="same"),
    ]

    up_stack = [
        Conv2DTranspose(512, (4, 4), strides=2, padding="same"),
        Conv2DTranspose(512, (4, 4), strides=2, padding="same"),
        Conv2DTranspose(512, (4, 4), strides=2, padding="same"),
        Conv2DTranspose(256, (4, 4), strides=2, padding="same"),
        Conv2DTranspose(128, (4, 4), strides=2, padding="same"),
        Conv2DTranspose(64, (4, 4), strides=2, padding="same"),
    ]

    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Concatenate()([x, skip])

    x = Conv2DTranspose(3, (4, 4), strides=2, padding="same", activation="tanh")(x)
    return Model(inputs=inputs, outputs=x)

# Build the Discriminator (PatchGAN)
def build_discriminator():
    inp = Input(shape=[256, 256, 3], name="input_image")
    tar = Input(shape=[256, 256, 3], name="target_image")
    x = Concatenate()([inp, tar])

    x = Conv2D(64, (4, 4), strides=2, padding="same")(x)
    x = LeakyReLU()(x)
    x = Conv2D(128, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(256, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(512, (4, 4), strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(1, (4, 4), strides=1, padding="same")(x)

    return Model(inputs=[inp, tar], outputs=x)

# Training loop
def train(dataset, epochs, generator, discriminator, gen_optimizer, disc_optimizer):
    original_images, cartoonized_images = dataset
    batch_size = 16

    for epoch in range(epochs):
        for i in range(0, len(original_images), batch_size):
            originals = original_images[i:i+batch_size]
            cartoonized = cartoonized_images[i:i+batch_size]
            
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                fake_images = generator(originals, training=True)

                real_output = discriminator([originals, cartoonized], training=True)
                fake_output = discriminator([originals, fake_images], training=True)

                gen_loss = tf.reduce_mean(tf.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output))
                l1_loss = tf.reduce_mean(tf.abs(cartoonized - fake_images))
                gen_total_loss = gen_loss + (100 * l1_loss)

                disc_real_loss = tf.reduce_mean(tf.losses.binary_crossentropy(tf.ones_like(real_output), real_output))
                disc_fake_loss = tf.reduce_mean(tf.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output))
                disc_loss = disc_real_loss + disc_fake_loss

            gen_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
            disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
            disc_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

        print(f"Epoch {epoch + 1}: Gen Loss = {gen_total_loss.numpy()}, Disc Loss = {disc_loss.numpy()}")

# Main script
if __name__ == "__main__":
    generator = build_generator()
    discriminator = build_discriminator()
    gen_optimizer = Adam(2e-4, beta_1=0.5)
    disc_optimizer = Adam(2e-4, beta_1=0.5)

    dataset = load_dataset("dataset/normal", "dataset/cartoon")
    
    # Répertoire pour sauvegarder les modèles
    save_dir = "saved_models"
    
    train(
        dataset,
        epochs=10,
        generator=generator,
        discriminator=discriminator,
        gen_optimizer=gen_optimizer,
        disc_optimizer=disc_optimizer,
        save_dir=save_dir
    )


MemoryError: Unable to allocate 384. KiB for an array with shape (256, 256, 3) and data type float16